# Get data from fedeo_ceda using EODAG

This tutorial will show you how to use EODAG to search and download data from the [fedeo_ceda](https://fedeo.ceda.ac.uk/) provider.

In [ ]:
from eodag import EODataAccessGateway, setup_logging

setup_logging(2)  # 0: nothing, 1: only progress bars, 2: INFO, 3: DEBUG
dag = EODataAccessGateway()
dag.set_preferred_provider("fedeo_ceda")

## List product types

list all the product types available from the fedeo_ceda provider.

In [ ]:
product_types = dag.list_product_types(provider="fedeo_ceda")
product_types

## Search data

With one of the product types listed above, you can search for data.

In [ ]:
fedeo_ceda_params = {
    "product_types": "AATSR_ENS_L2_V2.6",
    "start": "2006-01-01",
    "end": "2008-12-31",
    "format": "netCDF",
}

products = dag.search(
    provider="fedeo_ceda",
    **fedeo_ceda_params
)
products

## Open dataset with `to_xarray` from [eodag-cube](https://github.com/CS-SI/eodag-cube) and plot over a map using `cartopy`

In [ ]:
# Get XarrayDict
xd = products[2].to_xarray()
xd

In [ ]:
# Dataset from XarrayDict first value
ds = next(iter(xd.values()))
ds

In [ ]:
from datetime import datetime, UTC
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# data and coordinates from dataset
lats = ds["latitude"].values.ravel()
lons = ds["longitude"].values.ravel()
cct = ds["orbit"].values.ravel()

dt = datetime.strptime(ds.date_created, "%Y%m%dT%H%M%SZ").replace(tzinfo=UTC)
title = dt.isoformat()

fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})

# filled contour plot
tcf = ax.tricontourf(lons, lats, cct)
ax.set_title(title)
ax.coastlines()

# colorbar
cbar = plt.colorbar(tcf, ax=ax, shrink=0.7, label="orbit")

# gridlines
gridlines = ax.gridlines(draw_labels=True)
gridlines.right_labels = False
gridlines.top_labels = False

plt.show()

## Download data

You can download one or all of the products found during the search step.

In [ ]:
# Download one product
product = products[2]
download_info = dag.download(product)
download_info

# Download all products
download_infos = dag.download_all(products)
download_infos